In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [4]:
import pandas as pd

# The URL from your code
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Pandas can read directly from the URL!
spacex_df = pd.read_csv(URL)

# Now check the DataFrame
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [9]:
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values.
# In addition, add Launch site name as a popup label

# We iterate through each row in the launch_sites DataFrame
for index, site in launch_sites_df.iterrows():
    # Get the coordinates and name for each site
    coordinate = [site['Lat'], site['Long']]
    site_name = site['Launch Site']
    
    # 1. Create and add a folium.Circle for the launch site
    folium.Circle(
        location=coordinate,
        radius=1000,
        color='#000000',
        fill=True,
        fill_color='#3186cc',  # Added a fill color to make it visible
        popup=folium.Popup(site_name, parse_html=True)
    ).add_to(site_map)
    
    # 2. Create and add a folium.Marker with a custom DivIcon
    folium.Marker(
        location=coordinate,
        icon=folium.DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            # Use an f-string to insert the site name into the HTML
            html=f'<div style="font-size: 12; color:#d35400;"><b>{site_name}</b></div>'
        )
    ).add_to(site_map)

# After running this cell, display the map in the next cell to see the results
site_map

In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [11]:
marker_cluster = MarkerCluster()


In [12]:
# Apply a lambda function to check the value of `class` column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Check the first few rows to verify
print(spacex_df[['class', 'marker_color']].head())

   class marker_color
0      0          red
1      0          red
2      0          red
3      0          red
4      0          red


In [13]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was succeded or failed,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    
    # Create the Marker object
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        # Use the 'marker_color' column to set the icon color
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        # Add a popup to show the launch site on click
        popup=record['Launch Site']
    )
    
    # Add the marker to the marker_cluster
    marker_cluster.add_child(marker)

# Display the map
site_map

In [14]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [19]:
# 1. Define the coordinates
#    (We'll use the variable names from your marker code)
coastline_lat = 28.56367
coastline_lon = -80.57163
closest_coastline_coordinate = [coastline_lat, coastline_lon]

# 2. Define the distance
#    (This assumes you've already defined 'calculate_distance', 
#     'launch_site_lat', and 'launch_site_lon')
distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

print(f"Coordinates: {closest_coastline_coordinate}")
print(f"Distance: {distance:.2f} KM")

Coordinates: [28.56367, -80.57163]
Distance: 0.51 KM


In [20]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property

distance_marker = folium.Marker(
    location=closest_coastline_coordinate,  # Use the coordinate of the closest coastline point
    icon=folium.DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        # Use an f-string to insert the formatted distance variable into the HTML
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance:10.2f} KM</b></div>'
    )
)

# Add the new marker to the map
distance_marker.add_to(site_map)

# Display the map
site_map

In [27]:
# 1. Define the new coastline coordinates from your image
new_coastline_lat = 28.56406
new_coastline_lon = -80.56807
new_closest_coastline_coordinate = [new_coastline_lat, new_coastline_lon]

# 2. Recalculate the distance
#    (This assumes 'launch_site_lat', 'launch_site_lon', 
#     and the 'calculate_distance' function are still in memory)
new_distance = calculate_distance(launch_site_lat, launch_site_lon, new_coastline_lat, new_coastline_lon)

print(f"New Coordinates: {new_closest_coastline_coordinate}")
print(f"New Distance: {new_distance:.2f} KM")

# 3. Create and add the new distance marker
new_distance_marker = folium.Marker(
    location=new_closest_coastline_coordinate,
    icon=folium.DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{new_distance:10.2f} KM</b></div>'
    )
)
new_distance_marker.add_to(site_map)

# 4. Create and add the new PolyLine
#    (Assumes 'launch_site_coordinate' is still in memory: [launch_site_lat, launch_site_lon])
new_coordinates_for_line = [launch_site_coordinate, new_closest_coastline_coordinate]

new_line = folium.PolyLine(
    locations=new_coordinates_for_line, 
    weight=1,
    color='red'  # Make this line red to see the difference
)
new_line.add_to(site_map)

# Display the map (it will now show both lines)
site_map

New Coordinates: [28.56406, -80.56807]
New Distance: 0.86 KM


In [28]:
# 1. Define Coordinates
# --- West Coast Launch Site: Vandenberg (VAFB SLC-4E) ---
launch_site_lat = 34.632834
launch_site_lon = -120.610745
launch_site_coordinate = [launch_site_lat, launch_site_lon]

# --- Point of Interest: Lompoc, CA ---
poi_lat = 34.6391
poi_lon = -120.4579
poi_coordinate = [poi_lat, poi_lon]
poi_name = "Lompoc, CA"

# 2. Calculate Distance
distance_to_poi = calculate_distance(launch_site_lat, launch_site_lon, poi_lat, poi_lon)
print(f"Distance from VAFB to Lompoc: {distance_to_poi:.2f} KM")

# 3. Create a marker with distance to the city
distance_marker = folium.Marker(
    location=poi_coordinate,
    icon=folium.DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{poi_name}<br>{distance_to_poi:10.2f} KM</b></div>'
    )
)

# 4. Draw a line between the marker to the launch site
line_to_poi = folium.PolyLine(
    locations=[launch_site_coordinate, poi_coordinate], 
    weight=1,
    color='purple'  # Using a new color
)

# 5. Add to map
distance_marker.add_to(site_map)
line_to_poi.add_to(site_map)

# Display the map
site_map

Distance from VAFB to Lompoc: 14.01 KM
